In [1]:
%load_ext autoreload
%autoreload 2
#from lib.multitasking_lib import *

In [2]:
from lib.multitasking_lib import *
cache_type ='backtest_uni'

my_cache = cache_state('backtest_uni')

In [8]:
######################
# Initiate Backtest  #
######################
def store_algo(algo, algo_name="default"):
    redis_conn.hset("algos",algo_name, algo)
    algo_f = open("algo/"+algo_name+".txt", "w")
    algo_f.write(algo)
    algo_f.close()
    
#my_cache.flushall()
def backtest_unitest(stocks, freq='1T',algo_name = 'haikin_1_new', fromDate='2019-4-8', toDate='2019-4-9', mode='quick'):
    stocks = stocks
    sl = 1
    target = 1
    qty = 10
    backtest_msg={'stock':stocks,'sl':sl,'target':target,'qty':qty,'algo':algo_name,'fromDate':fromDate,'toDate':toDate,'freq':freq, 'mode':mode}

    # Step 4: Done is set to 0: Backtest is in progress, will be resetted by backtest job
    redis_conn.set('done'+cache_type,0)
    # Step 5: Send the msg to backtest thread to initiate the back test
    #pdebug(json.dumps(backtest_msg))
    redis_conn.publish('kite_simulator'+cache_type,json.dumps(backtest_msg))
    
##########################
# Unit Test              #
##########################

open('log/freedom.log','r+')
#log_file.close()

logger.setLevel(10)
loggerT.setLevel(21)
my_cache.set('done'+cache_type,1)
my_cache.xtrim('msgBufferQueue'+cache_type,0,False)
my_cache.set('last_id_msg'+cache_type, 0)


try:
    if backtest_manager.job.is_alive():
        backtest_manager.job.terminate()
        time.sleep(0.1)
except:
    pass
finally:
    backtest_manager = threadManager(cache_type, ["kite_simulator","ohlc_tick_handler","order_handler"], [kite_simulator, ohlc_tick_handler, order_handler])

#redis_conn.publish('ohlc_tick_handler'+cache_type,'start')
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
print(jobs)


True
['kite_ticker_handlerlive', 'order_notification_handlerlive', 'ohlc_tick_handlerbacktest_web', 'kite_simulatorbacktest_web', 'order_handlerbacktest_web', 'order_handlerlive', 'ohlc_tick_handlerlive']
[]


2020-05-25 22:50:18,842 INFO:	Starting Handler: kite_simulator
2020-05-25 22:50:18,853 INFO:	Starting Handler: ohlc_tick_handler
2020-05-25 22:50:18,856 INFO:	Starting Handler: order_handler
2020-05-25 22:50:18,908 DEBUG:	kite_simulator: 1
2020-05-25 22:50:19,115 DEBUG:	ohlc_tick_handler(backtest_uni) - INIT: 1
2020-05-25 22:50:19,118 DEBUG:	order_handler(backtest_uni): 1
2020-05-25 22:50:19,130 ERROR:	kite_simulator: Invalid msg: 1
2020-05-25 22:50:19,168 ERROR:	Error in order handler
2020-05-25 22:50:22,679 DEBUG:	kite_simulator: {"stock": ["TCS", "WIPRO", "INFRATEL"], "sl": 1, "target": 1, "qty": 10, "algo": "haikin_1_new", "fromDate": "2019-4-8", "toDate": "2019-4-9", "freq": "1T", "mode": "quick"}
2020-05-25 22:50:22,687 DEBUG:	Trade_init: TCS
2020-05-25 22:50:22,718 INFO:	Reset Cache for: TCSbacktest_uni
2020-05-25 22:50:22,754 INFO:	TCSbacktest_uni=>{'stock': 'TCS', 'qty': '0', 'SL %': '0.0', 'TP %': '0.0', 'amount': '0', 'price': '0.0', 'P&L': '0.0', 'P&L %': '0.0', 'Total P&L'

In [9]:
backtest_unitest(['TCS','WIPRO','INFRATEL'])

In [5]:
################################
###         Stop Trade       ###
################################
pinfo(cache_type)
my_cache.xtrim('msgBufferQueue'+cache_type,0,False)
my_cache.delete('msgBufferQueue'+cache_type)
my_cache.set('TCSbacktest_uniTICK',pd.DataFrame().to_json())

if backtest_manager.job.is_alive():
    backtest_manager.job.terminate()
time.sleep(1)
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
my_cache.set('last_id_msg',0)
jobs
jobs = []

2020-05-25 22:48:07,177 INFO:	backtest_uni


False
['kite_ticker_handlerlive', 'order_notification_handlerlive', 'ohlc_tick_handlerbacktest_web', 'kite_simulatorbacktest_web', 'order_handlerbacktest_web', 'order_handlerlive', 'ohlc_tick_handlerlive']


In [ ]:
pd.read_json(my_cache.get('TCSbacktest_uniTrade'))

In [ ]:
redis_conn.pubsub_channels()

In [ ]:
backtest_unitest(['TCS'])

In [ ]:
backtest_unitest(['WIPRO','TCS'])

In [ ]:
from lib.charting_lib import *
cache_type = 'backtest_uni'
chart_type ='haikin'

symbol='TCS'
my_cache = cache_state(cache_type)

#######  Hack for freq  ############
my_cache.setValue('TCS','freq','1T')
#####################################
dfohlc = my_cache.getOHLC(symbol)
print(dfohlc.tail())
trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')

render_charts(dfohlc, trade_df, symbol, chart_type)

In [ ]:
cache_state('backtest_uni').getValue()

## Live Trade Debugging

In [ ]:
live = threadManager('live', ["ohlc_tick_handler"], [ohlc_tick_handler])


In [ ]:
live_cache.setValue('WIPRO','mode','paper')

In [ ]:
live.job.terminate()
live_cache.set('WIPROliveTICK', pd.DataFrame().to_json(orient='columns'))
live_cache.setValue('WIPRO','last_processed',0)
live_cache.getValue('WIPRO')

In [ ]:
#live_cache.getOHLC('WIPRO') 
print(live_cache.xrange('msgBufferQueuelive')[-2:])
print(datetime.fromtimestamp(int(live_cache.xrange('msgBufferQueuelive')[-1][0].split('-')[0] )/1000))


In [ ]:
live_cache.getOHLC('WIPRO').tail()

In [7]:
my_cache.getOHLC('TCS')

""


## Quick Backtest

In [6]:
###########################################################
###                  Quick Backtest                     ###
###########################################################
from talib import MACD, MACDEXT, RSI, BBANDS, MACD, AROON, STOCHF, ATR, OBV, ADOSC, MINUS_DI, PLUS_DI, ADX, EMA, SMA
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR, ROC, MIN, MAX, MINMAX

ohlc_get = lambda df, key: df.iloc[-1][key]
def HAIKINASI(ohlc_data_df):
    REF = lambda key, i: ohlc_get(ohlc_data_df.shift(i), key)
    
    OPEN  = ohlc_data_df['open']
    HIGH  = ohlc_data_df['high']
    LOW   = ohlc_data_df['low']
    CLOSE = ohlc_data_df['close']
    
    haOPEN  = (OPEN.shift(1) + CLOSE.shift(1))/2
    haHIGH  = pd.DataFrame([HIGH,OPEN,CLOSE]).max(axis = 0, skipna = True)
    haLOW   = pd.DataFrame([LOW,OPEN,CLOSE]).min(axis = 0, skipna = True)
    haCLOSE = (OPEN+HIGH+LOW+CLOSE)/4
    
    return (haOPEN, haHIGH, haLOW, haCLOSE)


################ Load Data ###################
from lib.charting_lib import *
cache_type = 'backtest_uni'
symbol='TCS'
my_cache = cache_state(cache_type)

###  Set Freq for Algo
my_cache.setValue('TCS','freq','1T')
dfohlc = my_cache.getOHLC(symbol)
#print(dfohlc.head())

### Load Data
ohlc_data_temp = dfohlc
pd.set_option('display.max_rows', None)
################ Initialize Helper Functions ###################
trade_df1 = pd.DataFrame()
OPEN = ohlc_data_temp['open']
CLOSE = ohlc_data_temp['close']
HIGH = ohlc_data_temp['high']
LOW = ohlc_data_temp['low']
#VOLUME = ohlc_data_temp['volume']

(haOPEN, haHIGH, haLOW, haCLOSE) = HAIKINASI(ohlc_data_temp)

TIME = ohlc_data_temp.index.minute+ohlc_data_temp.index.hour*60

#REF = lambda df, i: df.iloc[-i-1]
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a,b: ROC(a, b) >= 0.1
TREND_DOWN = lambda a,b: ROC(a, b) <= -0.1
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)


def BUY(CLOSE, x, trade_df1):
    #global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["buy"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    return trade_df1

def SELL(CLOSE, x, trade_df1):
    #global trade_df1
    tmp_df = pd.DataFrame()
    tmp_df["sell"] = CLOSE[x]
    trade_df1 = trade_df1.append(tmp_df)
    return trade_df1
    
################# Write Algo For Backtest Here ###################

roc = ROC(CLOSE, 10)
bbh, bbm, bbl = BBANDS(CLOSE)

delbb = bbh - bbl

#sell = (REF(roc,0) > 0.1) & (REF(haOPEN, 0) > REF(haCLOSE,0)) & (REF(haOPEN, 1) < REF(haCLOSE,1))
#buy = (REF(roc,0) < -0.1) & (REF(haOPEN, 0) < REF(haCLOSE,0)) & (REF(haOPEN, 1) > REF(haCLOSE,1))
buy = delbb > 2.1 * REF(delbb, 3)
sell = delbb < 0.4 * REF(delbb, 3)

logic = buy | sell

summary_df = trade_df1.copy(deep=True)
#summary_df['haopen'] = haOPEN[ logic ]
#summary_df['haclose'] = haCLOSE[ logic ]
summary_df['roc'] = ROC(CLOSE, 30)[logic]

#print(summary_df.sort_index())
#print(summary_df.sort_index().tail(10))
#print(ROC(CLOSE,3))
#################### Render Charts ###############################

trade_df1 = SELL(CLOSE, sell, trade_df1)
trade_df1 = BUY(CLOSE, buy, trade_df1)

#print(trade_df1.sort_index())
my_cache.setCache(symbol+cache_type+'Trade',trade_df1)
#my_cache.pushTrade(symbol, trade_df1)

chart_type ='haikin'
trade_df = pd.read_json(redis_conn.get(symbol+cache_type+'Trade'), orient='columns')


render_charts(ohlc_data_temp, trade_df, symbol, chart_type)

KeyError: 'open'

In [ ]:

state = 'WAIT'
profit = 0
for index, row in trade_df.sort_index().fillna(0).iterrows():
    if state == 'WAIT':
        if row['sell']!=0:
            #print("S: ->: {}".format(row['sell']))
            state = 'SELL'
            profit = profit + row['sell']
        elif row['buy']!=0:
            #print("B: ->: {}".format(row['buy']))
            state = 'BUY'
            profit = profit - row['buy']
    elif state == 'BUY':
        if row['sell']!=0:
            #print("S: <-: {}".format(row['sell']))
            state = 'WAIT'
            profit = profit + row['sell']
    elif state == 'SELL':
        if row['buy']!=0:
            #print("B: <-: {}".format(row['buy']))
            state = 'WAIT'
            profit = profit - row['buy']
    
if state == 'BUY':
    profit = profit + row['buy']
elif state == 'SELL':
    profit = profit - row['sell']
    
print("Total Profit: {}".format(profit))

In [ ]:
bbh, bbm, bbl = BBANDS(CLOSE)

print(pd.DataFrame(data={'bbl':bbl, 'bbm':bbm, 'bbh':bbh, 'del':(bbh-bbl)/bbl*100}))

In [ ]:
(bbh-bbl)/bbl*100

In [ ]:
cache_type = 'backtest_web'
symbol='TCS'
my_cache = cache_state(cache_type)
my_cache.getTrades(symbol)

my_cache.get('TCSbacktest_webTrades')

In [ ]:
my_cache.getValue()

In [ ]:
temp_file = pd.HDFStore("data/kite_cache.h5", mode="r")
temp_file.keys()

## Redis Console

In [4]:
from lib.multitasking_lib import *
#from lib.data_model_lib import *
import json
web_cache = cache_state('backtest_web')
live_cache = cache_state('live')
uni_cache = cache_state('backtest_uni')

pd.set_option('display.max_rows', None)

In [ ]:
live_cache.pubsub_channels()

In [4]:
live_cache.xread({'notificationQueuelivenew':0}, block=2000, count=5000)

[]

In [ ]:
live_cache.remove()

In [ ]:
df = uni_cache.getValue()
df

In [ ]:
live_cache.getOHLC('TCS')

In [ ]:
trades_df = live_cache.getTrades('TCS')
trades_df

In [ ]:
(total_profit, max_loss, max_profit, total_win, total_loss, max_winning_streak, max_loosing_streak, trade_log_df) = trade_analysis_raw(trades_df)

In [ ]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from datetime import datetime as dt
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from lib.multitasking_lib import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

### GetData

In [ ]:
logger.setLevel(logging.DEBUG)
stock = 'WIPRO'
fromDate = '2019-4-22'
toDate  = '2019-4-24'
freq = 'minute'
mydate = fromDate
exchange = 'NSE'
ohlc_data = {}
ohlc_data[stock] = getData(stock, fromDate, toDate, exchange, freq, False, stock)

In [ ]:
ohlc_data[stock].head()

# Kite Connect

In [17]:
from kiteconnect import KiteConnect
kite_api_key = 'b2w0sfnr1zr92nxm'
live_cache.set('KiteAPIKey',kite_api_key)
access_token = live_cache.get('access_token')
print(access_token)
print(live_cache.get('KiteAPIKey'))

kite = KiteConnect(api_key=kite_api_key)
kite.set_access_token(access_token)

2307bLkt0EWdWt1DfTmSx9AxRg91l4V8
b2w0sfnr1zr92nxm


In [ ]:
kite.access_token

In [ ]:
kite.profile()

In [ ]:
cancel_order('CIPLA')

In [ ]:
orders_df = pd.DataFrame(kite.orders())
orders_df[['order_id','status','tradingsymbol','transaction_type','quantity','tag','price']]

In [ ]:
live_cache.getValue()

In [ ]:
position_df = pd.DataFrame(kite.positions()['net'])
position_df

In [20]:
pd.DataFrame(kite.positions()['net']).columns

Index(['tradingsymbol', 'exchange', 'instrument_token', 'product', 'quantity',
       'overnight_quantity', 'multiplier', 'average_price', 'close_price',
       'last_price', 'value', 'pnl', 'm2m', 'unrealised', 'realised',
       'buy_quantity', 'buy_price', 'buy_value', 'buy_m2m', 'sell_quantity',
       'sell_price', 'sell_value', 'sell_m2m', 'day_buy_quantity',
       'day_buy_price', 'day_buy_value', 'day_sell_quantity', 'day_sell_price',
       'day_sell_value'],
      dtype='object')

In [ ]:
orders_df = pd.DataFrame()
for msg in live_cache.xrange('notificationQueuelivenew'):
    tmp_df = pd.DataFrame.from_dict(json.loads(msg[1]['data']), orient='index').transpose()
    orders_df = orders_df.append(tmp_df)
    #print(msg)
orders_df[['order_id', 'status', 'tradingsymbol','order_type','transaction_type','quantity', 'average_price']]

In [ ]:
orders_df.columns

In [ ]:
live_cache.getValue().columns

In [ ]:
kite.margins()

#### Starting Live Trade for a stock

In [ ]:
symbol = 'INFY'
live_cache.add(symbol)
token = int(live_cache.hmget('eq_token',symbol)[0])
live_cache.publish('live_trade_handlerlive', json.dumps({'cmd':'add','value':[token],'mode':'ltp'}))

live_cache.setValue(symbol,'qty','1')
live_cache.setValue(symbol,'SL %','0.4')
live_cache.setValue(symbol,'TP %','1')
live_cache.setValue(symbol,'algo','haikin_1_new')
live_cache.setValue(symbol,'freq','1T')
live_cache.setValue(symbol,'last_processed',datetime.now().timestamp())
live_cache.setValue(symbol,'mode','paper')

In [ ]:
symbol = 'WIPRO'
live_cache.setValue(symbol,'last_processed',datetime.now().timestamp())

In [ ]:
datetime.now().timestamp()

In [ ]:
symbol = 'INFY'
live_cache.setValue(symbol,'mode','PAUSE')

In [ ]:
live_cache.getValue(symbol)

### Kite Ticker Handler Manipulation

In [ ]:
#live_cache.publish('live_trade_handlerlive','profile')
live_cache.publish('live_trade_handlerlive','status')

In [ ]:
live_cache.publish('kite_ticker_handlerlive','INIT')

In [ ]:
live_cache.publish('kite_ticker_handlerlive','CLOSE')

In [ ]:
live_cache.publish('kite_ticker_handlerlive','START')

In [ ]:
live_cache.publish('kite_ticker_handlerlive','profile')

In [ ]:
live_cache.smembers('live_token')

In [ ]:
values = list(map(int,live_cache.smembers('ticker_list')))
values

In [ ]:
live_cache.publish('kite_ticker_handlerlive',json.dumps({'cmd':'add','value':values,'mode':'ltp'}))

In [ ]:
live_cache.publish('kite_ticker_handlerlive',json.dumps({'cmd':'remove','value':[1510401],'mode':'quote'}))


In [ ]:
pd.DataFrame(kite.orders())

#### TickerList Manipulaitons

In [ ]:
live_cache.sadd('ticker_list','2953217')
live_cache.sadd('ticker_list','969473')

live_cache.sadd('ticker_list','1510401')

In [ ]:
values = list(map(int,live_cache.smembers('ticker_list')))
print(values)
live_cache.hmget('eq_token', values)

In [ ]:
live_cache.srem('ticker_list',1510401)

In [ ]:
live_cache.sismember('ticker_list',4268801)

In [ ]:
live_cache.delete('ticker_list')

In [ ]:
live_cache.getValue()

### Get Kite Ticker Status

In [ ]:
## Get Status of Kite
live_cache.get('Kite_Status')

### OrderHandler Monitor and Manipulation

In [ ]:
# Pause Ordering
live_cache.publish('order_handlerlive', 'pause')

In [ ]:
# Resume Ordering
live_cache.publish('order_handlerlive', 'resume')

In [ ]:
# Buy
live_cache.publish('order_handlerlive',json.dumps({'cmd':'buy','symbol':'WIPRO','price':180,'qty':1}))

In [ ]:
# Sell
live_cache.publish('order_handlerlive',json.dumps({'cmd':'sell','symbol':'WIPRO','price':190,'qty':1}))

In [ ]:
# Cancel
live_cache.publish('order_handlerlive',json.dumps({'cmd':'cancel','symbol':'WIPRO'}))

In [ ]:
live_cache.publish('order_handlerlive',json.dumps({'cmd':'getOrders'}))

In [ ]:
## Cancel All the orders
live_cache.publish('order_handlerlive',json.dumps({'cmd':'cancelAll'}))

In [ ]:
#from lib.kite_wrapper_lib import *
cancel_order(kite,['CIPLA'])

In [ ]:
buy_limit('WIPRO',188.5,1)

In [ ]:
sell_limit('WIPRO',188.7,1)

In [ ]:
live_cache.getOHLC('WIPRO').tail(10)

In [ ]:
kite.cancel_order(kite.VARIETY_REGULAR, 200511002924441, parent_order_id=None)

In [ ]:
kite.convert_position(kite.EXCHANGE_NSE, 'WIPRO', transaction_type, position_type, quantity, old_product, new_product)


In [ ]:
kite.modify_order(kite.VARIETY_REGULAR, 200511003019322, parent_order_id=None, quantity=None, price=187.4, order_type=None, trigger_price=None, validity=None, disclosed_quantity=None)


In [ ]:
msg = {'data': '[{"tradable": true, "mode": "quote", "instrument_token": 969473, "last_price": 184.7, "last_quantity": 5, "average_price": 187.07, "volume": 4393242, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 185.0, "high": 189.95, "low": 184.1, "close": 184.0}, "change": 0.38043478260868946}, {"tradable": true, "mode": "quote", "instrument_token": 2953217, "last_price": 1935.3, "last_quantity": 25, "average_price": 1939.47, "volume": 2449395, "buy_quantity": 0, "sell_quantity": 0, "ohlc": {"open": 1909.0, "high": 1951.4, "low": 1909.0, "close": 1893.4}, "change": 2.2129502482306886}]'}

In [ ]:
msg['data']

In [ ]:
val = json.loads(msg['data'])

##### Reset state for tradejob

In [ ]:
live_cache.setValue('WIPRO','state','SCANNING')

### Live Cache State Monitor and manipulation

In [7]:
live_cache.getValue()

,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq,order_id
0,WIPRO,1,0.4,1.5,185.1,185.1,-0.75,-0.40683482506102525,184.05000000000004,99.43273905996762,...,0.0,0,185.1,1589886717.871,SCANNING,pause,haikin_1_new_roc,1T,minute,200519003064351
1,BPCL,1,0.4,1.5,292.95,0,-3.0500000000000114,-1.0304054054054093,0.75,0.2560163850486431,...,0.0,0,299.0,1589886717.871,SCANNING,pause,haikin_1_new_roc,1T,day,0
2,INFRATEL,1,0.4,1.5,212.6,212.6,0,0,7.599999999999994,3.5747883349012204,...,0.0,215.789,220.6,1589886717.871,SCANNING,live,haikin_1_new_roc,1T,day,200519001474585


In [ ]:
#Modify cache table
stock_key = 'CIPLA'
#live_cache.add(stock_key, True) #TODO: While adding stock to table use True
live_cache.setValue(stock_key, 'algo', 'haikin_1_new')
live_cache.setValue(stock_key, 'freq', '1T')
live_cache.setValue(stock_key, 'qty', 1)
live_cache.setValue(stock_key, 'SL %', 1)
live_cache.setValue(stock_key, 'TP %', 1)
live_cache.setValue(stock_key, 'P&L', 0)
live_cache.setValue(stock_key, 'Total P&L', 0)
live_cache.setValue(stock_key, 'price', 0)
live_cache.setValue(stock_key, 'hdf_freq', 'minute')
live_cache.getValue()

In [ ]:
live_cache.setValue('WIPRO', 'mode', 'paper')

In [ ]:
#live_cache.remove('AXISBANK')
live_cache.getValue()

In [ ]:
live_cache.remove()

#### Get token for Stock symbol

In [ ]:
live_cache.hmget('eq_token',['CIPLA','INFY'])

In [ ]:
live_cache.hmget('eq_token',['TCS','WIPRO','AXISBANK'])

#### Peek into OHLC buffer

In [9]:
live_cache.getOHLC('INFRATEL').tail(5)

,close,high,low,open
2020-05-21 10:32:00,207.50,208.35,207.45,208.25
2020-05-21 10:33:00,207.30,207.70,207.10,207.50
2020-05-21 10:34:00,207.50,207.65,207.25,207.30
2020-05-21 10:35:00,206.65,207.50,206.55,207.40
2020-05-21 10:36:00,207.10,207.30,206.70,206.85


#### msgBufferQueue Monitoring

In [8]:
peek_msg_queue = live_cache.xrange('msgBufferQueuelive')[-1]
print(peek_msg_queue)
datetime.fromtimestamp(int(peek_msg_queue[0].split('-')[0] )/1000)

('1590037606838-0', {'data': '[{"tradable": true, "mode": "quote", "instrument_token": 134657, "last_price": 308.1, "last_quantity": 10, "average_price": 305.98, "volume": 2669100, "buy_quantity": 654327, "sell_quantity": 552836, "ohlc": {"open": 310.0, "high": 311.4, "low": 301.6, "close": 308.9}, "change": -0.2589834898025104}, {"tradable": true, "mode": "quote", "instrument_token": 7458561, "last_price": 207.1, "last_quantity": 10, "average_price": 207.59, "volume": 6337952, "buy_quantity": 931509, "sell_quantity": 1200065, "ohlc": {"open": 204.0, "high": 209.65, "low": 204.0, "close": 200.85}, "change": 3.1117749564351507}, {"tradable": true, "mode": "quote", "instrument_token": 969473, "last_price": 187.6, "last_quantity": 1, "average_price": 186.05, "volume": 1595068, "buy_quantity": 972665, "sell_quantity": 719224, "ohlc": {"open": 183.4, "high": 187.75, "low": 183.4, "close": 184.95}, "change": 1.4328196809948666}]'})


datetime.datetime(2020, 5, 21, 10, 36, 46, 838000)

In [15]:
live_cache.xrange('notificationQueuelivenew')

[('1590119523620-0',
  {'data': '{"placed_by": "YM1755", "order_id": "200522000319311", "exchange_order_id": "1300000000825811", "parent_order_id": null, "status": "OPEN", "status_message": null, "status_message_raw": null, "order_timestamp": "2020-05-22 09:22:03", "exchange_update_timestamp": "2020-05-22 09:22:03", "exchange_timestamp": "2020-05-22 09:22:03", "variety": "regular", "exchange": "NSE", "tradingsymbol": "WIPRO", "instrument_token": 969473, "order_type": "LIMIT", "transaction_type": "BUY", "validity": "DAY", "product": "MIS", "quantity": 1, "disclosed_quantity": 0, "price": 187.8, "trigger_price": 0, "average_price": 0, "filled_quantity": 0, "pending_quantity": 1, "cancelled_quantity": 0, "market_protection": 0, "meta_raw": null, "meta": {}, "tag": "freedom_v2", "tags": ["freedom_v2"], "guid": "11327XJvZwilDmKOpq", "account_id": "YM1755", "unfilled_quantity": 0, "app_id": 11327, "checksum": ""}'}),
 ('1590119723375-0',
  {'data': '{"placed_by": "YM1755", "order_id": "20052

In [ ]:
datetime.fromtimestamp(1589428580968/1000)

In [16]:
live_cache.getOHLC('INFRATEL').tail()

,close,high,low,open
2020-05-19 15:07:00,219.45,219.85,219.40,219.40
2020-05-19 15:08:00,219.00,219.50,219.00,219.45
2020-05-19 15:09:00,218.75,219.50,218.75,219.00
2020-05-19 15:48:00,218.75,218.75,218.75,218.75
2020-05-19 16:41:00,218.75,218.75,218.75,218.75


##### Monitor: OHLC Tick Handler

In [ ]:
val = live_cache.get('last_id_msg')
print(val)
datetime.fromtimestamp(int(val.split('-')[0] )/1000)

In [ ]:
# Timestamp to ID
timestamp_to_id = lambda x:str(int(float(x)*1000))+'-'+str(0)
id_val = timestamp_to_id(live_cache.getValue('WIPRO','last_processed'))
print(id_val)
live_cache.xread({'msgBufferQueuelive':id_val}, count=1)

In [ ]:
# Peek into the last msg processed by tick handler
print(live_cache.getValue('WIPRO','last_processed'))
datetime.fromtimestamp(float(live_cache.getValue('WIPRO','last_processed')))

In [ ]:
## Ticker job monitoring
live_cache.get('tick_count')

In [ ]:
datetime.fromtimestamp(1589535456664/1000)

In [ ]:
msgBufferDump = live_cache.xrange('msgBufferQueuelive')

In [ ]:
fp = open('log/ticker_log.log',mode='w')

In [ ]:
fp.write(json.dumps(msgBufferDump))

### Update Redis with Instrument list

##### Download fresh data and store in cache

In [ ]:
instruments_df = pd.DataFrame(kite.instruments())

In [ ]:
instruments_df.to_hdf('data/instruments.h5',key='instruments')

In [ ]:
instruments_df.columns

In [ ]:
eq_nse = instruments_df.loc[(instruments_df['exchange']=='NSE') & (instruments_df['segment']=='NSE')  & (instruments_df['instrument_type']=='EQ'), ['instrument_token','tradingsymbol']]

In [ ]:
eq_nse.loc[eq_nse['tradingsymbol'].isin(['TCS','WIPRO']),'instrument_token'].values

##### Use cache

In [ ]:
instruments_df = pd.read_hdf('data/instruments.h5',key='instruments')

In [ ]:
instruments_df

In [ ]:
eq_nse = instruments_df.loc[(instruments_df['exchange']=='NSE') & (instruments_df['segment']=='NSE')  & (instruments_df['instrument_type']=='EQ'), ['instrument_token','tradingsymbol']]

In [ ]:
eq_nse.loc[eq_nse['tradingsymbol'].isin(['TCS','WIPRO']),:].values

eq_nse.set_index('instrument_token')

In [ ]:
live_cache.delete('eq_token')

In [ ]:
eq_nse.head().to_dict(orient='records')

In [ ]:
eq_nse.head().set_index('tradingsymbol').transpose().to_dict(orient='records')

In [ ]:
eq_nse.head().set_index('instrument_token').transpose().to_dict(orient='records')[0]

##### Update redis with tokens

In [ ]:
live_cache.hmset('eq_token',eq_nse.set_index('instrument_token').transpose().to_dict(orient='records')[0])
live_cache.hmset('eq_token',eq_nse.set_index('tradingsymbol').transpose().to_dict(orient='records')[0])

In [ ]:
live_cache.hmget('eq_token',['TCS','WIPRO'])

In [ ]:
live_cache.hgetall('eq_token')

In [ ]:
live_cache.hmget('eq_token',[2953217, 969473])

In [1]:
import pandas as pd

In [4]:
df = pd.read_hdf('data/kite_cache_day_backup.h5')

ValueError: key must be provided when HDF5 file contains multiple datasets.

In [2]:
import pandas as pd
kite_cache_path = 'data/kite_cache.h5'
temp_file = pd.HDFStore(kite_cache_path, mode="r")
#rDate = temp_file.get(key).tail(1).index

In [25]:
temp_file.keys()

['/NSE',
 '/minute/minute/BHARTIARTL',
 '/minute/NSE/3MINDIA',
 '/minute/NSE/AARTIIND',
 '/minute/NSE/ABB',
 '/minute/NSE/ABFRL',
 '/minute/NSE/ACC',
 '/minute/NSE/ACE',
 '/minute/NSE/ADANIENT',
 '/minute/NSE/ADANIPORTS',
 '/minute/NSE/ADANIPOWER',
 '/minute/NSE/AIAENG',
 '/minute/NSE/AJANTPHARM',
 '/minute/NSE/AKZOINDIA',
 '/minute/NSE/ALBK',
 '/minute/NSE/ALKEM',
 '/minute/NSE/ALLCARGO',
 '/minute/NSE/AMARAJABAT',
 '/minute/NSE/AMBUJACEM',
 '/minute/NSE/ANANTRAJ',
 '/minute/NSE/APOLLOHOSP',
 '/minute/NSE/APOLLOTYRE',
 '/minute/NSE/ARVIND',
 '/minute/NSE/ASAHIINDIA',
 '/minute/NSE/ASHOKLEY',
 '/minute/NSE/ASIANPAINT',
 '/minute/NSE/ASTRAZEN',
 '/minute/NSE/ATFL',
 '/minute/NSE/ATUL',
 '/minute/NSE/AUBANK',
 '/minute/NSE/AUROPHARMA',
 '/minute/NSE/AUTOAXLES',
 '/minute/NSE/AVANTIFEED',
 '/minute/NSE/AXISBANK',
 '/minute/NSE/BAJAJ-AUTO',
 '/minute/NSE/BAJAJELEC',
 '/minute/NSE/BAJAJFINSV',
 '/minute/NSE/BAJAJHLDNG',
 '/minute/NSE/BAJFINANCE',
 '/minute/NSE/BALKRISIND',
 '/minute/NSE/BAL

In [14]:
df1 = pd.read_hdf(temp_file, key='/minute/NSE/WIPRO')

In [3]:
keys = ['BHARTIARTL', '3MINDIA', 'AARTIIND', 'ABB', 'ABFRL', 'ACC', 'ACE', 'ADANIENT', 'ADANIPORTS', 'ADANIPOWER', 'AIAENG', 'AJANTPHARM', 'AKZOINDIA', 'ALBK', 'ALKEM', 'ALLCARGO', 'AMARAJABAT', 'AMBUJACEM', 'ANANTRAJ', 'APOLLOHOSP', 'APOLLOTYRE', 'ARVIND', 'ASAHIINDIA', 'ASHOKLEY', 'ASIANPAINT', 'ASTRAZEN', 'ATFL', 'ATUL', 'AUBANK', 'AUROPHARMA', 'AUTOAXLES', 'AVANTIFEED', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJELEC', 'BAJAJFINSV', 'BAJAJHLDNG', 'BAJFINANCE', 'BALKRISIND', 'BALRAMCHIN', 'BANCOINDIA', 'BANKBARODA', 'BANKBEES', 'BANKINDIA', 'BATAINDIA', 'BBTC', 'BEL', 'BEML', 'BERGEPAINT', 'BFUTILITIE', 'BHARATFIN', 'BHARATFORG', 'BHARTIARTL', 'BHEL', 'BIOCON', 'BLISSGVS', 'BLKASHYAP', 'BLUEDART', 'BOSCHLTD', 'BPCL', 'BRITANNIA', 'BSE', 'CADILAHC', 'CAMLINFINE', 'CANBK', 'CANFINHOME', 'CAPACITE', 'CASTROLIND', 'CDSL', 'CEATLTD', 'CENTRUM', 'CENTURYPLY', 'CENTURYTEX', 'CEREBRAINT', 'CESC', 'CGCL', 'CGPOWER', 'CHENNPETRO', 'CHOLAFIN', 'CIPLA', 'CNOVAPETRO', 'COALINDIA', 'COCHINSHIP', 'COFFEEDAY', 'COLPAL', 'CONCOR', 'COROMANDEL', 'COX&KINGS', 'CRISIL', 'CROMPTON', 'CUB', 'CUMMINSIND', 'CYIENT', 'DAAWAT', 'DABUR', 'DALMIASUG', 'DBCORP', 'DCBBANK', 'DCMSHRIRAM', 'DEN', 'DHAMPURSUG', 'DHFL', 'DIAMONDYD', 'DISHTV', 'DIVISLAB', 'DIXON', 'DLF', 'DMART', 'DREDGECORP', 'DRREDDY', 'ECLERX', 'EICHERMOT', 'EIDPARRY', 'EIHOTEL', 'ELECTCAST', 'EMAMILTD', 'ENDURANCE', 'ENGINERSIN', 'EQUITAS', 'ERIS', 'EROSMEDIA', 'ESCORTS', 'ESTER', 'EXIDEIND', 'FEDERALBNK', 'FEL', 'FINCABLES', 'FORTIS', 'FRETAIL', 'GAIL', 'GATI', 'GDL', 'GEPIL', 'GESHIP', 'GET&D', 'GHCL', 'GICHSGFIN', 'GICRE', 'GILLETTE', 'GLAXO', 'GLENMARK', 'GLOBUSSPR', 'GMBREW', 'GMRINFRA', 'GNA', 'GNFC', 'GODFRYPHLP', 'GODREJAGRO', 'GODREJCP', 'GODREJIND', 'GODREJPROP', 'GOLDBEES', 'GPPL', 'GRANULES', 'GRASIM', 'GREAVESCOT', 'GSFC', 'GSKCONS', 'GSPL', 'GUJALKALI', 'GUJFLUORO', 'GUJGASLTD', 'HATHWAY', 'HAVELLS', 'HBLPOWER', 'HCL-INSYS', 'HCLTECH', 'HDFC', 'HDFCAMC', 'HDFCBANK', 'HDFCLIFE', 'HEIDELBERG', 'HEROMOTOCO', 'HEXAWARE', 'HGS', 'HIKAL', 'HINDALCO', 'HINDCOPPER', 'HINDPETRO', 'HINDUNILVR', 'HINDZINC', 'HONAUT', 'HSCL', 'HSIL', 'HUDCO', 'IBULHSGFIN', 'ICICIBANK', 'ICICIGI', 'ICICINIFTY', 'ICICIPRULI', 'IDBI', 'IDEA', 'IDFC', 'IDFCFIRSTB', 'IEX', 'IFCI', 'IGL', 'INDHOTEL', 'INDIACEM', 'INDIANB', 'INDIGO', 'INDUSINDBK', 'INFIBEAM', 'INFRATEL', 'INFY', 'INOXLEISUR', 'INOXWIND', 'INTELLECT', 'IOB', 'IOC', 'IPCALAB', 'IRB', 'ITC', 'ITDCEM', 'JAGRAN', 'JAICORPLTD', 'JAMNAAUTO', 'JAYAGROGN', 'JAYSREETEA', 'JETAIRWAYS', 'JINDALSTEL', 'JINDWORLD', 'JISLJALEQS', 'JKCEMENT', 'JKPAPER', 'JKTYRE', 'JMFINANCIL', 'JPASSOCIAT', 'JSLHISAR', 'JSWENERGY', 'JSWSTEEL', 'JUBILANT', 'JUBLFOOD', 'JUSTDIAL', 'JYOTHYLAB', 'KAJARIACER', 'KALPATPOWR', 'KANSAINER', 'KARURVYSYA', 'KEC', 'KEI', 'KILITCH', 'KIRIINDUS', 'KOTAKBANK', 'KOTAKNIFTY', 'KSCL', 'KTKBANK', 'L&TFH', 'LALPATHLAB', 'LIBERTSHOE', 'LICHSGFIN', 'LINCOLN', 'LIQUIDBEES', 'LOVABLE', 'LT', 'LUPIN', 'M&M', 'M&MFIN', 'M50', 'MAGMA', 'MAHINDCIE', 'MAHLOG', 'MAHSEAMLES', 'MAJESCO', 'MANALIPETC', 'MANAPPURAM', 'MANGTIMBER', 'MANINDS', 'MANINFRA', 'MARICO', 'MARKSANS', 'MARUTI', 'MASFIN', 'MASTEK', 'MCDOWELL-N', 'MCX', 'MEGH', 'MFSL', 'MGL', 'MIDHANI', 'MINDACORP', 'MINDAIND', 'MINDTREE', 'MMTC', 'MOIL', 'MOLDTKPAC', 'MOREPENLAB', 'MOTHERSUMI', 'MPHASIS', 'MRF', 'MRPL', 'MUKANDLTD', 'MUNJALAU', 'MUTHOOTFIN', 'NATCOPHARM', 'NATIONALUM', 'NAUKRI', 'NAVINFLUOR', 'NAVNETEDUL', 'NBCC', 'NCC', 'NESTLEIND', 'NETWORK18', 'NFL', 'NH', 'NHPC', 'NIACL', 'NIFTY 100', 'NIFTY 200', 'NIFTY 50', 'NIFTY AUTO', 'NIFTY BANK', 'NIFTY COMMODITIES', 'NIFTY ENERGY', 'NIFTY FIN SERVICE', 'NIFTY FMCG', 'NIFTY INFRA', 'NIFTY IT', 'NIFTY JUNIOR', 'NIFTY MEDIA', 'NIFTY METAL', 'NIFTY MIDCAP 100', 'NIFTY MIDCAP 50', 'NIFTY MNC', 'NIFTY PHARMA', 'NIFTY PSU BANK', 'NIFTY REALTY', 'NIFTY SMALL 100', 'NIFTYBEES', 'NIITTECH', 'NLCINDIA', 'NMDC', 'NOCIL', 'NRBBEARING', 'NTPC', 'OBEROIRLTY', 'OFSS', 'OIL', 'OMAXE', 'ONGC', 'ORIENTBANK', 'ORIENTCEM', 'PAGEIND', 'PAPERPROD', 'PARAGMILK', 'PATELENG', 'PCJEWELLER', 'PEL', 'PERSISTENT', 'PETRONET', 'PFC', 'PFIZER', 'PGHH', 'PHILIPCARB', 'PHOENIXLTD', 'PIDILITIND', 'PIIND', 'PNB', 'PNBGILTS', 'PNBHOUSING', 'POLYPLEX', 'POWERGRID', 'PRAJIND', 'PRESTIGE', 'PTC', 'PURVA', 'PVR', 'QUICKHEAL', 'RADICO', 'RADIOCITY', 'RAJESHEXPO', 'RALLIS', 'RAMCOCEM', 'RAMCOIND', 'RAMKY', 'RAYMOND', 'RBLBANK', 'RCF', 'RECLTD', 'RELAXO', 'RELCAPITAL', 'RELIANCE', 'RELINFRA', 'REPCOHOME', 'RICOAUTO', 'RIIL', 'RKFORGE', 'RPOWER', 'SAIL', 'SAKUMA', 'SALASAR', 'SALZERELEC', 'SANGHIIND', 'SANGHVIMOV', 'SANOFI', 'SAREGAMA', 'SBILIFE', 'SBIN', 'SCHAEFFLER', 'SCHAND', 'SCHNEIDER', 'SCI', 'SELAN', 'SEQUENT', 'SHARDAMOTR', 'SHREECEM', 'SHRIRAMCIT', 'SIEMENS', 'SIMPLEXINF', 'SINTEX', 'SIS', 'SJVN', 'SKFINDIA', 'SNOWMAN', 'SOBHA', 'SOLARINDS', 'SOUTHBANK', 'SPARC', 'SPTL', 'SREINFRA', 'SRF', 'SRTRANSFIN', 'STAR', 'STCINDIA', 'STRTECH', 'SUNDARMFIN', 'SUNDRMFAST', 'SUNPHARMA', 'SUNTECK', 'SUNTV', 'SUPREMEIND', 'SUVEN', 'SUZLON', 'SYNDIBANK', 'SYNGENE', 'TAKE', 'TALWALKARS', 'TATACHEM', 'TATACOFFEE', 'TATACOMM', 'TATAELXSI', 'TATAGLOBAL', 'TATAINVEST', 'TATAMOTORS', 'TATAMTRDVR', 'TATAPOWER', 'TATASPONGE', 'TATASTEEL', 'TBZ', 'TCI', 'TCS', 'TECHM', 'TEJASNET', 'TEXMOPIPES', 'TEXRAIL', 'THERMAX', 'THOMASCOOK', 'THYROCARE', 'TIMETECHNO', 'TINPLATE', 'TITAN', 'TNPETRO', 'TNPL', 'TORNTPHARM', 'TORNTPOWER', 'TRENT', 'TRIDENT', 'TRIGYN', 'TRIVENI', 'TTKPRESTIG', 'TV18BRDCST', 'TVSMOTOR', 'TWL', 'UBL', 'UCOBANK', 'UJJIVAN', 'ULTRACEMCO', 'UNIONBANK', 'UNIVCABLES', 'UPL', 'USHAMART', 'VEDL', 'VGUARD', 'VIPCLOTHNG', 'VIPIND', 'VOLTAS', 'VTL', 'WABCOINDIA', 'WALCHANNAG', 'WELCORP', 'WELENT', 'WELSPUNIND', 'WHIRLPOOL', 'WINDMACHIN', 'WIPRO', 'WOCKPHARMA', 'WONDERLA', 'YESBANK', 'ZEEL']

temp_file1 = pd.HDFStore('data/kite_cache_sample.h5', mode="w")
for key in keys:
    hash1 = '/minute/NSE/'+key
    hash2 = '/day/NSE/'+key
    df1 = pd.read_hdf(temp_file, key=hash1)
    df2 = pd.read_hdf(temp_file, key=hash2)
    
    df1.tail(10000).to_hdf(temp_file1, hash1)
    df2.tail(1000).to_hdf(temp_file1, hash2)
    
    #print(key)
    
    temp_file1.close()

/usr/local/lib/python3.7/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: '3MINDIA'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/usr/local/lib/python3.7/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'BAJAJ-AUTO'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/usr/local/lib/python3.7/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'COX&KINGS'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_n

/usr/local/lib/python3.7/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'NIFTY MNC'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/usr/local/lib/python3.7/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'NIFTY PHARMA'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/usr/local/lib/python3.7/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'NIFTY PSU BANK'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_at

In [4]:
pd.HDFStore('data/kite_cache_sample.h5', mode="r").keys()

['/minute/NSE/BPCL',
 '/minute/NSE/INFRATEL',
 '/minute/NSE/TCS',
 '/minute/NSE/WIPRO',
 '/day/NSE/BPCL',
 '/day/NSE/INFRATEL',
 '/day/NSE/TCS',
 '/day/NSE/WIPRO']

In [6]:
pd.read_hdf('data/kite_cache_sample.h5', key='/minute/NSE/WIPRO')

,close,high,low,open,volume
date,,,,,
2019-03-22 11:20:00,259.80,259.80,259.75,259.75,1202
2019-03-22 11:21:00,259.80,259.95,259.65,259.80,2975
2019-03-22 11:22:00,260.00,260.00,259.75,259.80,5195
2019-03-22 11:23:00,259.95,260.00,259.95,260.00,2723
2019-03-22 11:24:00,259.90,260.00,259.85,259.95,2462
...,...,...,...,...,...
2019-05-03 15:25:00,291.15,291.20,290.85,290.90,45200
2019-05-03 15:26:00,291.15,291.15,291.10,291.15,23576
2019-05-03 15:27:00,291.05,291.15,290.95,291.15,41539


In [8]:
temp_file1.close()